In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

Learning goal: To consider effective implementations of nearest neighbor search. 

Consider the nearest neighbor search problem: Given a dataset of n objects $X = {x_1,...,x_n}$ and a query object $q$, we want to find the object $x∗ \in X$ that minimizes the distance $d(q, x)$, that is,

$$d(q, x∗) \le d(q, x) \quad \forall x \in X. \quad (1)$$

Assume that computing the distance function d is very expensive. Assume now that we are able to define another distance function $d_{LB}$, which is a lower bound of distance $d$. This means that for all pairs of objects x and y it should be 

$$d_{LB}(x, y) \le d(x, y) \quad (2) $$

Furthermore, assume that computing $d_{LB}$ is significantly more efficient than computing d.

(a) Write pseudocode of an algorithm for the nearest neighbor search using distance d.

```
def d(x, y):
    # expensive distance function
    return <d_value>

def NearestNeighborSearch(X, q):
    min_distance = inf
    nearest_neighbor = None

    for each object x_i in X:
        dist = d(x_i, q)
        ff dist < min_distance:
            min_distance = dist
            nearest_neighbor = x
    return nearest_neighbor
```

(b) Explain how to use the lower-bound distance dLB to speed up the search algorithm of the previous part. Write pseudocode for the modified search algorithm.

We can use the Lower Bound distance to enable Early Stopping 

If the lower-bound distance $d_{LB}(x, q)$ for an object x is larger than d(y, q), where y is another object, then x cannot be nearer to q than y is due to the inequality 

$$ d(x, q) \ge d_{LB}(x, q) > d(y, q)  $$
Therefore, we can skip computing d(x, q). 

How to incorporate the lower-bound distance into the search algorithm?:

First, we compute the lower-bound distance for all objects in X. Then, we sort the objects in X according to their lower-bound distances, and call it X_sorted. Finally, we use X_sorted to perform the nearest neighbor search. If the lower-bound distance of an object x_i during the search is larger than the current true distance of the current nearest neighbor, then we can stop the search, since we know that all remaining objects in X_sorted are farther away from q than the current nearest neighbor due to the inequality above

The pseudocode: 

```
def d(x, y):
    # expensive distance function
    return <d_value>

def d_LB(x, y):
    # cheap lower bound distance function
    return <d_LB_value>

def NearestNeighborSearchLB(X, q):
    """
    Arguments:
        X: Data set of N objects {x1,...,xn}
        q: Query object
    Return:
        x*: Object in X that is nearest to q based on distance d  
    """
        
    # Initialize an array to store the lower-bound distances
    X_dLB = [] 
    
    # compute lower-bound distances
    for each object x_i in X:
        dist_LB = d_LB(x_i, q)
        X_dLB.append(dist_LB)
    
    # sort the objects in X according to their lower-bound distances
    X_zip_dLB = zip(X, X_dLB)
    X_zip_dLB_sorted = sorted(X_zip_dLB, key=x[1]) # sort X by dLB

    min_distance = inf
    nearest_neighbor = None

    # We perform the search on the sorted X based on dLB
    for each tuple x_i, dLB in X_zip_dLB_sorted:
        if dLB > min_distance:
            break 
            # early stopping, since all remaining objects are 
            # farther away than the current nearest neighbor
        
        # Important: only calculate d if dLB <= min_distance
        dist = d(x_i, q)
        if dist < min_distance:
            min_distance = dist
            nearest_neighbor = x_i
    
    return nearest_neighbor
```

(c) What is a desirable property for the lower-bound distance dLB to be as
effective as possible for the modified algorithm? Explain why.

Three properties are desirable for the lower-bound distance dLB to be as effective as possible for the modified algorithm:

1. The lower-bound distance should be as close as possible to the true distance. Good $d_{LB}$ will help as early as possible pruning data points that are far away from the query point.

2. The lower-bound distance should be as cheap as possible to compute. Otherwise, the cost of computing the lower-bound distance will be too high and will not help to speed up the search.

3. The lower-bound distance should be unique for each object as much as possible. Otherwise, the sorting step becomes trivial and we cannot carry out much pruning. 